In [4]:
using Turing
using Random
using ForwardDiff
using LimberJack
using CSV
using NPZ
using FITSIO
using Plots
using LinearAlgebra
using BlockArrays
using Distributed
using DataFrames

┌ Info: Precompiling LimberJack [6b86205d-155a-4b14-b82d-b6a149ea78f2]
└ @ Base loading.jl:1423
┌ Warning: Package LimberJack does not have NPZ in its dependencies:
│ - If you have LimberJack checked out for development and have
│   added NPZ as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LimberJack
└ Loading NPZ into LimberJack from project dependency, future warnings for LimberJack are suppressed.


In [5]:
# Add four processes to use for sampling.
addprocs(4)

# Initialize everything on all the processes.
# Note: Make sure to do this after you've already loaded Turing,
#       so each process does not have to precompile.
#       Parallel sampling may fail silently if you do not do this.
@everywhere using Turing, LimberJack

      From worker 4:	┌ Warning: Package LimberJack does not have NPZ in its dependencies:
      From worker 4:	│ - If you have LimberJack checked out for development and have
      From worker 4:	│   added NPZ as a dependency but haven't updated your primary
      From worker 4:	│   environment's manifest file, try `Pkg.resolve()`.
      From worker 4:	│ - Otherwise you may need to report an issue with LimberJack
      From worker 4:	└ Loading NPZ into LimberJack from project dependency, future warnings for LimberJack are suppressed.
      From worker 5:	┌ Warning: Package LimberJack does not have NPZ in its dependencies:
      From worker 5:	│ - If you have LimberJack checked out for development and have
      From worker 5:	│   added NPZ as a dependency but haven't updated your primary
      From worker 5:	│   environment's manifest file, try `Pkg.resolve()`.
      From worker 5:	│ - Otherwise you may need to report an issue with LimberJack
      From worker 5:	└ Loading NPZ into Lim

In [6]:
desgs_data = npzread("/home/jaime/PhD/LimberJack.jl/data/cl_DESgc__2_DESwl__3.npz")["cl"]
desgs_data = transpose(desgs_data)[1:39]
desgs_cov = npzread("/home/jaime/PhD/LimberJack.jl/data/cov_DESgc__2_DESwl__3_DESgc__2_DESwl__3.npz")["cov"]
desgs_cov = desgs_cov[1:39, 1:39]
desgs_cov = Symmetric(Hermitian(desgs_cov))
desgs_err = (view(desgs_cov, diagind(desgs_cov)))[1:39].^0.5
desgs_nzs = FITS("/home/jaime/PhD/LimberJack.jl/data/y1_redshift_distributions_v1.fits")
desgs_nz = read(desgs_nzs["nz_source_mcal"], "BIN2")
desgs_zs = read(desgs_nzs["nz_source_mcal"], "Z_MID");

desgg_data = npzread("/home/jaime/PhD/LimberJack.jl/data/cl_DESgc__2_DESgc__2.npz")["cl"]
desgg_data = transpose(desgg_data)[1:39]
desgg_cov = npzread("/home/jaime/PhD/LimberJack.jl/data/cov_DESgc__2_DESgc__2_DESgc__2_DESgc__2.npz")["cov"]
desgg_cov = desgg_cov[1:39, 1:39]
desgg_cov = Symmetric(Hermitian(desgg_cov))
desgg_err = (view(desgg_cov, diagind(desgg_cov)))[1:39].^0.5
desgg_nzs = FITS("/home/jaime/PhD/LimberJack.jl/data/y1_redshift_distributions_v1.fits")
desgg_nz = read(desgg_nzs["nz_source_mcal"], "BIN2")
desgg_zs = read(desgg_nzs["nz_source_mcal"], "Z_MID");

ell = npzread("/home/jaime/PhD/LimberJack.jl/data/cl_DESgc__2_DESwl__3.npz")["ell"]
ell = [Int(floor(l)) for l in ell]
path = "/home/jaime/PhD/LimberJack.jl/data/";

In [40]:
@model function DESgg_model(data)
    Ωm ~ Uniform(0.2, 0.3)
    h ~ Uniform(0.6, 0.8)
    s8 ~ Uniform(0.7, 1.0)
    cosmology = LimberJack.Cosmology(Ωm, 0.05, h, 0.96, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    tg = NumberCountsTracer(cosmology, desgg_zs, desgg_nz, 2.)
    predictions = [angularCℓ(cosmology, tg, tg, ℓ) for ℓ in ell]
    data ~ MvNormal(predictions, desgg_cov)
end;

iterations = 10
step_size = 0.05
samples_per_step = 5
cores = 4

# Start sampling.

folname = string("DESgg_test_", "stpsz_", step_size, "_smpls_", samples_per_step)
if isdir(folname)
    println("Folder already exists")
    if isfile("chain.jls")
        println("Restarting from past chain")
        past_chain = read("chain.jls", Chains)
        new_chain = sample(DESgg_model(desgg_data), HMC(step_size, samples_per_step),
                           MCMCDistributed(), iterations, cores, progress=true;
                           save_state=true, resume_from=past_chain)
    end
else
    mkdir(folname)
    println("Created new folder")
    new_chain = sample(DESgg_model(desgg_data), HMC(step_size, samples_per_step),
                       MCMCDistributed(), iterations, cores, progress=true; save_state=true)
end

info = describe(new_chain)[1]
fname_info = string("info.csv")
CSV.write(joinpath(folname, fname_info), info)


fname_jls = string("chain.jls")
write(joinpath(folname, fname_jls), new_chain)
    
fname_csv = string("chain.csv")
CSV.write(joinpath(folname, fname_csv), new_chain)

Folder already exists
Restarting from past chain
      From worker 2:	┌ Warning: The current proposal will be rejected due to numerical error(s).
      From worker 2:	│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
      From worker 2:	└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
      From worker 3:	┌ Warning: The current proposal will be rejected due to numerical error(s).
      From worker 3:	│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
      From worker 3:	└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
      From worker 4:	┌ Warning: The current proposal will be rejected due to numerical error(s).
      From worker 4:	│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
      From worker 4:	└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
      From worker 5:	┌ Warning: The current proposal will be rejected due to numerical error(s).
      From worker 5:	│   isfinite.(

Sampling (4 processes): 100%|███████████████████████████| Time: 0:00:03


      From worker 3:	└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
      From worker 4:	┌ Warning: The current proposal will be rejected due to numerical error(s).
      From worker 4:	│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
      From worker 4:	└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
      From worker 5:	┌ Warning: The current proposal will be rejected due to numerical error(s).
      From worker 5:	│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
      From worker 5:	└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
      From worker 2:	┌ Warning: The current proposal will be rejected due to numerical error(s).
      From worker 2:	│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
      From worker 2:	└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
      From worker 3:	┌ Warning: The current proposal will be rejected due to numerical

┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364
┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364
┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364
┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364
┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364
┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364
┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364
┌ Warning: timestamp of type Nothing unknown
└ @ MCMCChains /home/jaime/.julia/packages/MCMCChains/kWf0l/src/chains.jl:364


"DESgg_test_stpsz_0.05_smpls_5/chain.csv"

In [9]:
@model function DESgs_model(data)
    Ωm ~ Uniform(0.2, 0.3)
    h ~ Uniform(0.6, 0.8)
    s8 ~ Uniform(0.7, 1.0)
    cosmology = LimberJack.Cosmology(Ωm, 0.05, h, 0.96, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    tg = NumberCountsTracer(cosmology, desgg_zs, desgg_nz, 2.)
    ts = WeakLensing(cosmology, desgs_zs, desgs_nz)
    predictions = [angularCℓ(cosmology, tg, ts, ℓ) for ℓ in ell]
    data ~ MvNormal(predictions, desgg_cov)
end;

iterations = 10
step_size = 0.05
samples_per_step = 5
cores = 4

# Start sampling.

folname = string("DESgs_test_", "stpsz_", step_size, "_smpls_", samples_per_step)
if isdir(folname)
    println("Folder already exists")
    if isfile("chain.jls")
        println("Restarting from past chain")
        past_chain = read("chain.jls", Chains)
        new_chain = sample(DESgg_model(desgg_data), HMC(step_size, samples_per_step),
                           MCMCDistributed(), iterations, cores, progress=true;
                           save_state=true, resume_from=past_chain)
    end
else
    mkdir(folname)
    println("Created new folder")
    new_chain = sample(DESgg_model(desgg_data), HMC(step_size, samples_per_step),
                       MCMCDistributed(), iterations, cores, progress=true; save_state=true)
end

info = describe(new_chain)[1]
fname_info = string("info.csv")
CSV.write(joinpath(folname, fname_info), info)


fname_jls = string("chain.jls")
write(joinpath(folname, fname_jls), new_chain)
    
fname_csv = string("chain.csv")
CSV.write(joinpath(folname, fname_csv), new_chain)

┌ Warning: Only a single thread available: MCMC chains are not sampled in parallel
└ @ AbstractMCMC /home/jaime/.julia/packages/AbstractMCMC/oou1a/src/sample.jl:253


"Cl_DESgs_test_chain.csv"